In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.7 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 37.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [ ]:
pip install torch==1.9.1+cpu torchvision==0.10.1+cpu torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 175.4 MB 25 kB/s 
     |████████████████████████████████| 15.7 MB 21 kB/s 
     |████████████████████████████████| 1.9 MB 5.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1+cpu which is incompatible.


**Setup Sumarization Model**

In [ ]:
!pip install PyTorch

  ERROR: Failed building wheel for PyTorch
  Running setup.py clean for PyTorch
Failed to build PyTorch
    Running setup.py install for PyTorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-2ylo235l/pytorch_a575aa68173d4e2190abcf8c35ab7d35/setup.py'"'"'; __file__='"'"'/tmp/pip-install-2ylo235l/pytorch_a575aa68173d4e2190abcf8c35ab7d35/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ui93k_m4/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/PyTorch Check the logs for full command output.


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [ ]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Testing summarize a single article
https://finance.yahoo.com/news/bitcoin-price-prediction-bulls-eye-114742508.html

In [ ]:
url = "https://finance.yahoo.com/news/bitcoin-price-prediction-bulls-eye-114742508.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs[1].text

'After a bullish day for Bitcoin and the majors on Wednesday, it’s been a broadly bullish Wednesday morning.'

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:350]
ARTICLE = ' '.join(words)

In [ ]:
ARTICLE

' After a bullish day for Bitcoin and the majors on Wednesday, it’s been a broadly bullish Wednesday morning. At the time of writing, Bitcoin, BTC to USD, was up by 3.75% to $43,086.1. A mixed start to the day saw Bitcoin fall to an early morning low $41,432.0 before making a move. Steering clear of the first major support level at $40,687, Bitcoin rallied to a mid-morning current day high $43,860. Bitcoin broke through the first major resistance level at $42,489 and the 38.2% FIB of $41,592. The early rally saw Bitcoin also break through the second major resistance level at $43,449 before revising sub-$43,000 levels. It has been a mixed morning for the broader crypto market. At the time of writing, Polkadot was down by 1.61% to buck the morning trend. It’s been a bullish morning for the rest of the majors, however. Chainlink (+3.92%), Ethereum (+3.81%), and Litecoin (+4.11%) set the morning pace. Binance Coin (+1.39%), Bitcoin Cash SV (+2.04%), Cardano’s ADA (+0.84%), Crypto.com Coin 

In [ ]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')

In [ ]:
output = model.generate(
    input_ids, 
    max_length=55, 
    num_beams=5, 
    early_stopping=True
)

In [ ]:
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
summary

'Largest cryptocurrency has rallied to a mid-morning high. Chainlink, Ethereum, and Litecoin set the morning pace'

**Build Pipeline**

In [ ]:
monitored_tickers = ['BTC', 'ETH', 'AVAX-USD']


**Search for news using Google & Yahoo Finance**

In [ ]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [ ]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'AVAX-USD': ['/?sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQPAgE',
  '/search?q=yahoo+finance+AVAX-USD&tbm=nws&ie=UTF-8&gbv=1&sei=gFB2YcDmDfXgxgGR5JOAAw',
  '/search?q=yahoo+finance+AVAX-USD&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUIBygA',
  '/search?q=yahoo+finance+AVAX-USD&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUICSgC',
  '/search?q=yahoo+finance+AVAX-USD&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUICigD',
  'https://maps.google.com/maps?q=yahoo+finance+AVAX-USD&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUICygE',
  '/search?q=yahoo+finance+AVAX-USD&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUIDCgF',
  '/search?q=yahoo+finance+AVAX-USD&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjApuKu-eTzAhV1sDEKHRHyBDAQ_AUIDSgG',
  '/advanced_search',
  '/search?q=yahoo+finance+AVAX

In [ ]:
raw_urls['BTC']

['/?sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQOwgC',
 '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQPAgE',
 '/search?q=yahoo+finance+BTC&tbm=nws&ie=UTF-8&gbv=1&sei=f1B2YfH3Dp2QwbkPwZuX4Aw',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUIBygA',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUICSgC',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUICigD',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUICygE',
 'https://maps.google.com/maps?q=yahoo+finance+BTC&um=1&ie=UTF-8&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUIDCgF',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwixs6au-eTzAhUdSDABHcHNBcwQ_AUIDSgG',
 '/advanced_search',
 '/search?q=yahoo+finance+BTC&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwixs

**Remove unwanted URLS**

In [ ]:
import re

In [ ]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'AVAX-USD': ['https://finance.yahoo.com/news/24-old-gen-zer-became-100000997.html',
  'https://finance.yahoo.com/news/avalanche-avax-token-jumps-180m-181749941.html',
  'https://sg.finance.yahoo.com/news/hottest-crypto-coins-are-now-the-bitcoin-and-ether-alternatives-013823172.html',
  'https://markets.businessinsider.com/news/currencies/altcoin-outlook-cryptocurrencies-q4-polygon-boost-cro-solana-cardano-avalanche-2021-10',
  'https://finance.yahoo.com/news/bitcoin-testing-50-000-resistance-155820724.html',
  'https://finance.yahoo.com/news/market-wrap-bitcoin-trades-sideways-203648408.html',
  'https://finance.yahoo.com/news/defi-amm-sushiswap-builds-avalanche-130101149.html',
  'https://finance.yahoo.com/news/coinbase-launch-avax-trading-ahead-144417569.html',
  'https://finance.yahoo.com/news/avalanche-keep-defi-users-rush-010114732.html',
  'https://finance.yahoo.com/news/altcoins-rally-cryptocurrency-market-recovers-132636190.html'],
 'BTC': ['https://finance.yahoo.com/amphtml/n

**Search and Scrape Cleaned URLS**

In [ ]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [ ]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'AVAX-USD': ['24-year old Rahul Rai became a millionaire in May after BlockTower Capital acquired the fund he\'d co-founded. Rai now runs the BlockTower Gamma Point Market Neutral Fund. He believes avalanche is the ethereum network\'s closest rival and will "steal liquidity" from the bigger blockchain. Sign up here for our daily newsletter, 10 Things Before the Opening Bell. As cryptocurrencies become increasingly mainstream and more blockchain-based applications are springing up every day, competition is heating up between the different networks to host as much of that activity as possible. The ethereum network is the busiest by a long way, but 24-year old Rahul Rai, who became a millionaire when he sold a crypto fund he helped found, believes the avalanche blockchain has the most potential to take that number one spot. Rai grew up in India, worked as an analyst for Morgan Stanley in 2019 and founded crypto hedge fund Gamma Point Capital in 2021 together with Eash Aggarwal and Sanat 

In [ ]:
articles['BTC'][3]

"(Bloomberg) -- Most Read from Bloomberg Why Americans and Britons Are Rushing to Buy Idyllic Homes in Italy Cities' Answer to Sprawl? Go Wild. One of California’s Wealthiest Counties Could Run Out of Water Next Summer The Top Money Maker at Deutsche Bank Reaps Billions From Singapore Hamburg Is at the Heart of Germany’s Growing Dilemma Over China The second futures-backed U.S. Bitcoin exchange-traded fund made a slightly less conspicuous debut than the launch of the first such investment product three days ago. The Valkyrie Bitcoin Strategy ETF -- trading under the ticker BTF -- fell 2.8% from its inception price of $25 Friday. More than 3.1 million shares worth about $78 million changed hands during the session. Bitcoin, meanwhile, pared an earlier advance to fall as much as 4.3% to $60,011. The largest digital currency reached an all-time high of just under $67,000 on Wednesday. “There does seem to be a first mover advantage that plays out,” said Stephane Ouellette, chief executive 

**Summarise all Articles**

In [ ]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [ ]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'AVAX-USD': ['avalanche is closest rival to ethereum, he says. Rai grew up in India, worked as an analyst for Morgan Stanley in 2019',
  'We are aware of the issue and are working to resolve it.',
  'Cardano, Avalanche surge after big rallies for Bitcoin, Ether. Prices for photos with laser eyes are going gangbusters',
  'Markets to focus on network upgrades, scalability: Oanda. Here are six companies to watch in the fourth quarter',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.',
  'Exchange to list AVAX native token on Coinbase Pro. Coinbase remains the most popular exchange in the US ahead of FTX',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.'],
 'BTC': ['Retail investors may be better off just purchasing the asset. ProShares Bitcoin Strategy ETF made its debut on New Yo

In [ ]:
summaries['BTC']

['Retail investors may be better off just purchasing the asset. ProShares Bitcoin Strategy ETF made its debut on New York Stock Exchange',
 'New York-based firm is in talks for $1 billion investment. CFTC is looking into whether Polymarket is breaking rules',
 'ProShares will host a conference call with investors on Thursday. Open interest and volume on the CME were expected to rise',
 'Valkyrie Bitcoin Strategy ETF takes assets over $1 billion. VanEck is set to launch a similar fund next week',
 'We are aware of the issue and are working to resolve it.',
 'The coin has lost about 8% since reaching an all-time high. Two futures-backed U.S. Bitcoin ETFs took assets over $1 billion in two days',
 'The lawsuit claims SEC violated rules and hired private process service.',
 'We are aware of the issue and are working to resolve it.',
 'Chainlink leads the way with a 9.58% gain, followed by Bitcoin Cash.',
 'Valkyrie launches Bitcoin ETF on Nasdaq. Company expects at least two more crypto-ba

**Adding Sentiment Analysis**

In [ ]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
sentiment(summaries['BTC'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[{'label': 'NEGATIVE', 'score': 0.9965948462486267},
 {'label': 'NEGATIVE', 'score': 0.9558952450752258},
 {'label': 'POSITIVE', 'score': 0.9968835711479187},
 {'label': 'NEGATIVE', 'score': 0.9783180952072144},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.999359667301178},
 {'label': 'NEGATIVE', 'score': 0.99298095703125},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.6248753666877747},
 {'label': 'NEGATIVE', 'score': 0.9989308714866638}]

In [ ]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'AVAX-USD': [{'label': 'POSITIVE', 'score': 0.9923396110534668},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9695053100585938},
  {'label': 'POSITIVE', 'score': 0.860296905040741},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9983674883842468},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002}],
 'BTC': [{'label': 'NEGATIVE', 'score': 0.9965948462486267},
  {'label': 'NEGATIVE', 'score': 0.9558952450752258},
  {'label': 'POSITIVE', 'score': 0.9968835711479187},
  {'label': 'NEGATIVE', 'score': 0.9783180952072144},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.999359667301178},
  {'label': 'NEGATIVE', 'score': 0.99298095703125},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 

In [ ]:
print(summaries['BTC'][3], scores['BTC'][3]['label'], scores['BTC'][3]['score'])

Valkyrie Bitcoin Strategy ETF takes assets over $1 billion. VanEck is set to launch a similar fund next week NEGATIVE 0.9783180952072144


**Export Results to CSV**

In [1]:
#### Andrew, I couldnt run this project on my machine, and on Colab I didnt get the CSV file.

In [ ]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [ ]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['BTC',
  'Retail investors may be better off just purchasing the asset. ProShares Bitcoin Strategy ETF made its debut on New York Stock Exchange',
  'NEGATIVE',
  0.9965948462486267,
  'https://finance.yahoo.com/amphtml/news/bitcoin-hit-2-milestones-week-100000636.html'],
 ['BTC',
  'New York-based firm is in talks for $1 billion investment. CFTC is looking into whether Polymarket is breaking rules',
  'NEGATIVE',
  0.9558952450752258,
  'https://finance.yahoo.com/news/crypto-venue-bets-trump-jlo-135159112.html'],
 ['BTC',
  'ProShares will host a conference call with investors on Thursday. Open interest and volume on the CME were expected to rise',
  'POSITIVE',
  0.9968835711479187,
  'https://finance.yahoo.com/news/cme-volume-gets-pumped-bitcoin-155000606.html'],
 ['BTC',
  'Valkyrie Bitcoin Strategy ETF takes assets over $1 billion. VanEck is set to launch a similar fund next week',
  'NEGATIVE',
  0.9783180952072144,
  'https://finance.yahoo.com/news/second-u-bitcoin-etf-launche

In [ ]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [ ]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['BTC',
  'Retail investors may be better off just purchasing the asset. ProShares Bitcoin Strategy ETF made its debut on New York Stock Exchange',
  'NEGATIVE',
  0.9965948462486267,
  'https://finance.yahoo.com/amphtml/news/bitcoin-hit-2-milestones-week-100000636.html'],
 ['BTC',
  'New York-based firm is in talks for $1 billion investment. CFTC is looking into whether Polymarket is breaking rules',
  'NEGATIVE',
  0.9558952450752258,
  'https://finance.yahoo.com/news/crypto-venue-bets-trump-jlo-135159112.html'],
 ['BTC',
  'ProShares will host a conference call with investors on Thursday. Open interest and volume on the CME were expected to rise',
  'POSITIVE',
  0.9968835711479187,
  'https://finance.yahoo.com/news/cme-volume-gets-pumped-bitcoin-155000606.html'],
 ['BTC',
  'Valkyrie Bitcoin Strategy ETF takes assets over $1 billion. VanEck is set to launch a similar fund next week',
  'NEGATIVE',
  0.9783180952072144,
  'https

In [ ]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_output)

In [ ]:
df.head()

,0,1,2,3,4
0,Ticker,Summary,Label,Confidence,URL
1,BTC,Retail investors may be better off just purcha...,NEGATIVE,0.996595,https://finance.yahoo.com/amphtml/news/bitcoin...
2,BTC,New York-based firm is in talks for $1 billion...,NEGATIVE,0.955895,https://finance.yahoo.com/news/crypto-venue-be...
3,BTC,ProShares will host a conference call with inv...,POSITIVE,0.996884,https://finance.yahoo.com/news/cme-volume-gets...
4,BTC,Valkyrie Bitcoin Strategy ETF takes assets ove...,NEGATIVE,0.978318,https://finance.yahoo.com/news/second-u-bitcoi...


In [ ]:
df[2].value_counts()

NEGATIVE    16
POSITIVE    14
Label        1
Name: 2, dtype: int64